In [1]:
pip install azure-kusto-data

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azure-identity

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade azure-kusto-data

In [4]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

In [5]:
AAD_TENANT_ID = "common"
KUSTO_CLUSTER = "https://kvc-nng49g644r9bekcc3f.australiaeast.kusto.windows.net"
KUSTO_DATABASE = "MyDataBase"
kcsb = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
kcsb.authority_id = AAD_TENANT_ID
Kusto_client = KustoClient(kcsb)
query = """OpenCellH3Data
| where MCC == 204
| where isnotempty(H3_Medium)
| summarize arg_max(Updated, *) by RadioType, MCC, MNC, LAC
| summarize SignalStrength=avg(AvgSignal), dcount(LAC), UmtsCells = countif(RadioType == "UMTS"), LTE = countif(RadioType == "LTE") by H3_Medium
| extend H3_Hash_Polygon = geo_h3cell_to_polygon(H3_Medium)
| extend Features=pack(
"geometry", H3_Hash_Polygon,
"id", H3_Medium,
"properties",pack("value", UmtsCells),
"type", "Feature")"""

response = Kusto_client.execute(KUSTO_DATABASE, query)
df = dataframe_from_result_table(response.primary_results[0])
feature_list = df['Features'].to_list()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SBJJ28LPT to authenticate.


In [6]:
pip install geojson

Note: you may need to restart the kernel to use updated packages.


In [9]:
from geojson import FeatureCollection
feat_collection = FeatureCollection(feature_list)
import plotly.express as px

In [17]:
df.dropna(subset=['UmtsCells'], inplace=True)

fig = (px.choropleth_mapbox(
    df, 
    geojson=feat_collection, 
    locations='H3_Medium', 
    color='UmtsCells',
    color_continuous_scale="agsunset",
    mapbox_style='carto-positron',
    zoom=6.5,
    center={"lat": -6.200000, "lon": 106.816666}, 
    opacity=0.6,
    hover_data=['SignalStrength', 'dcount_LAC', 'UmtsCells', 'LTE'],
    width=700,
    height=800,
))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()